<a href="https://colab.research.google.com/github/shekhar-banerjee96/911callslist/blob/master/Amazon_product_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib  inline

import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings('ignore')

import sqlite3
import nltk
import string

from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer

from sklearn.metrics import confusion_matrix , classification_report , accuracy_score , roc_curve , auc

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import pickle

In [5]:
#conn = sqlite3.connect('C:/Users/HP-PC/DEV/Learnbay/Stats_ML/Dataset/database.sqlite')
conn = sqlite3.connect('/content/drive/MyDrive/database.sqlite')
conn

In [6]:
df = pd.read_sql_query("""select * from Reviews where Score <> 3 LIMIT 25000""",con=conn)
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [7]:
df.shape

(25000, 10)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Id                      25000 non-null  int64 
 1   ProductId               25000 non-null  object
 2   UserId                  25000 non-null  object
 3   ProfileName             25000 non-null  object
 4   HelpfulnessNumerator    25000 non-null  int64 
 5   HelpfulnessDenominator  25000 non-null  int64 
 6   Score                   25000 non-null  int64 
 7   Time                    25000 non-null  int64 
 8   Summary                 25000 non-null  object
 9   Text                    25000 non-null  object
dtypes: int64(5), object(5)
memory usage: 1.9+ MB


In [9]:
df['Score'].value_counts()

5    16890
4     4023
1     2495
2     1592
Name: Score, dtype: int64

In [10]:
def categorize(x) :
    return 0 if x < 3 else 1

df['Score'] = df['Score'].apply(lambda x : categorize(x))

In [11]:
df['Score'].value_counts()

1    20913
0     4087
Name: Score, dtype: int64

In [12]:
df_users = pd.read_sql_query("""sELECT Text, Score, UserId, count(*) FROM Reviews group by UserId Having count(*) > 1""",con=conn)
df_users.head()

,Text,Score,UserId,count(*)
0,Overall its just OK when considering the price...,2,#oc-R115TNMSPFT9I7,2
1,"My wife has recurring extreme muscle spasms, u...",5,#oc-R11D9D7SHXIJB9,3
2,This coffee is horrible and unfortunately not ...,1,#oc-R11DNU2NBKQ23Z,2
3,This will be the bottle that you grab from the...,5,#oc-R11O5J5ZVQE25C,3
4,I didnt like this coffee. Instead of telling y...,1,#oc-R12KPBODL2B5ZD,2


In [13]:
df_users[df_users['UserId']=='AZZNK89PXD006']

,Text,Score,UserId,count(*)
80663,In this package you get three varieties of pop...,5,AZZNK89PXD006,6


In [14]:
df_users['count(*)'].sum()

393063

# EDA

In [15]:
display = pd.read_sql_query("""

select * from reviews where Score !=3 and UserID ="AZZNK89PXD006"
ORDER BY ProductId""", conn)

display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,185765,B0009YX9UU,AZZNK89PXD006,Etude,1,1,4,1269648000,Fizzy cola fun,"I love Japanese candy, so this package was eag..."
1,152692,B0018CIPS8,AZZNK89PXD006,Etude,2,2,5,1319068800,Improved the health of my cat,My vet suggested that a grain-free diet would ...
2,84898,B001J3ZUQ6,AZZNK89PXD006,Etude,8,9,5,1269648000,Great oil-popping popcorn!,In this package you get three varieties of pop...
3,314009,B0033XV37K,AZZNK89PXD006,Etude,1,1,4,1273881600,Better than the movies themselves!,For around thirty bucks I was greeted at the d...
4,427048,B0039GMZZK,AZZNK89PXD006,Etude,1,1,5,1269648000,"Limited Edition, Unlimited Flavor!",Regular Chocolate Pocky is good on its own but...


In [16]:
# sorting data in an ascending order
sorted_data = df.sort_values('ProductId', axis=0, ascending=True, inplace=False,
                                       kind='quicksort', na_position='last')

In [17]:
sorted_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
22620,24750,2734888454,A13ISQV0U9GZIC,Sandikaye,1,1,0,1192060800,made in china,My dogs loves this chicken but its a product f...
22621,24751,2734888454,A1C298ITT645B6,Hugh G. Pritchard,0,0,1,1195948800,Dog Lover Delites,Our dogs just love them. I saw them in a pet ...
2547,2775,B00002NCJC,A13RRPGE79XFFH,reader48,0,0,1,1281052800,Flies Begone,We have used the Victor fly bait for 3 seasons...
2546,2774,B00002NCJC,A196AJHU9EASJN,Alex Chaffee,0,0,1,1282953600,thirty bucks?,Why is this $[...] when the same product is av...
1145,1244,B00002Z754,A3B8RCEI0FXFI6,B G Chase,10,10,1,962236800,WOW Make your own 'slickers' !,I just received my shipment and could hardly w...


In [18]:
sorted_data.duplicated().sum()

0

In [19]:
# In case you get any duplicate data, you have to drop it
final = sorted_data.drop_duplicates(subset={'UserId','ProfileName','Time','Text'}, keep='first', inplace=False)

final.shape

(23953, 10)

In [20]:
final.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
22620,24750,2734888454,A13ISQV0U9GZIC,Sandikaye,1,1,0,1192060800,made in china,My dogs loves this chicken but its a product f...
22621,24751,2734888454,A1C298ITT645B6,Hugh G. Pritchard,0,0,1,1195948800,Dog Lover Delites,Our dogs just love them. I saw them in a pet ...
2547,2775,B00002NCJC,A13RRPGE79XFFH,reader48,0,0,1,1281052800,Flies Begone,We have used the Victor fly bait for 3 seasons...
2546,2774,B00002NCJC,A196AJHU9EASJN,Alex Chaffee,0,0,1,1282953600,thirty bucks?,Why is this $[...] when the same product is av...
1145,1244,B00002Z754,A3B8RCEI0FXFI6,B G Chase,10,10,1,962236800,WOW Make your own 'slickers' !,I just received my shipment and could hardly w...


In [21]:
final = final[final['HelpfulnessNumerator'] <= final['HelpfulnessDenominator']]

In [22]:
final.shape

(23953, 10)

## Data Processing

In [23]:
# printing some sample random variables
sent_1 = final['Text'].values[1]
print(sent_1)
print("="*50)

sent_100 = final['Text'].values[100]
print(sent_100)
print("="*50)

sent_1000 = final['Text'].values[1000]
print(sent_1000)
print("="*50)

sent_6552 = final['Text'].values[6552]
print(sent_6552)
print("="*50)

sent_9000 = final['Text'].values[9000]
print(sent_9000)
print("="*50)

Our dogs just love them.  I saw them in a pet store and a tag was attached regarding them being made in China and it satisfied me that they were safe.
After reading the book "Deadly Feasts" I was horrified to learn what kinds of items may end up in my dogs' food! I started researching different dog foods and consistently received the same brand recommendations: Innova, Canidae, Wellness, Chicken Soup for the Dog Lover's Soul...etc. I initially settled on Canidae, but also tried Wellness on a recommendation. The Wellness produced loose stools in two of my foster poodles, so I no longer use that brand. The Canidae, however, is perfect for my multi dog household, which often includes dogs ranging from puppyhood to senior status. The ingredients are phenomenal and, as all reviewers have said, the food contains no corn, wheat or soy, all common allergens in dogs. Although I also use and would recommend Innova Evo dry dog food, Canidae costs $35 for 40 pounds and Innova costs about $50 for a

## regular Expression

In [24]:
import re

sent_1 = re.sub(r"http\S+","",sent_1)
sent_100 = re.sub(r"http\S+","",sent_100)
sent_1000 = re.sub(r"http\S+","",sent_1000)
sent_6552 = re.sub(r"http\S+","",sent_6552)
sent_9000 = re.sub(r"http\S+","",sent_9000)


print(sent_1, end ='\n\n')
print(sent_100, end ='\n\n')
print(sent_1000, end ='\n\n')
print(sent_6552, end ='\n\n')
print(sent_9000, end ='\n\n')

Our dogs just love them.  I saw them in a pet store and a tag was attached regarding them being made in China and it satisfied me that they were safe.

After reading the book "Deadly Feasts" I was horrified to learn what kinds of items may end up in my dogs' food! I started researching different dog foods and consistently received the same brand recommendations: Innova, Canidae, Wellness, Chicken Soup for the Dog Lover's Soul...etc. I initially settled on Canidae, but also tried Wellness on a recommendation. The Wellness produced loose stools in two of my foster poodles, so I no longer use that brand. The Canidae, however, is perfect for my multi dog household, which often includes dogs ranging from puppyhood to senior status. The ingredients are phenomenal and, as all reviewers have said, the food contains no corn, wheat or soy, all common allergens in dogs. Although I also use and would recommend Innova Evo dry dog food, Canidae costs $35 for 40 pounds and Innova costs about $50 for 

In [25]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(sent_1,'lxml')
text = soup.get_text()
print(text)
print('='*50)

soup = BeautifulSoup(sent_100 ,'lxml')
text = soup.get_text()
print(text)
print('='*50)

soup = BeautifulSoup(sent_1000,'lxml')
text = soup.get_text()
print(text)
print('='*50)

soup = BeautifulSoup(sent_6552,'lxml')
text = soup.get_text()
print(text)
print('='*50)


soup = BeautifulSoup(sent_9000,'lxml')
text = soup.get_text()
print(text)
print('='*50)

Our dogs just love them.  I saw them in a pet store and a tag was attached regarding them being made in China and it satisfied me that they were safe.
After reading the book "Deadly Feasts" I was horrified to learn what kinds of items may end up in my dogs' food! I started researching different dog foods and consistently received the same brand recommendations: Innova, Canidae, Wellness, Chicken Soup for the Dog Lover's Soul...etc. I initially settled on Canidae, but also tried Wellness on a recommendation. The Wellness produced loose stools in two of my foster poodles, so I no longer use that brand. The Canidae, however, is perfect for my multi dog household, which often includes dogs ranging from puppyhood to senior status. The ingredients are phenomenal and, as all reviewers have said, the food contains no corn, wheat or soy, all common allergens in dogs. Although I also use and would recommend Innova Evo dry dog food, Canidae costs $35 for 40 pounds and Innova costs about $50 for a

In [26]:
# remove sopecial characters

sent_1  =  re.sub('[^A-Za-z0-9]+',' ',sent_1)
print(sent_1)

Our dogs just love them I saw them in a pet store and a tag was attached regarding them being made in China and it satisfied me that they were safe 


In [27]:
stopwords = set(['br','M380', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

## All clean Up

import sys
!{sys.executable} -m pip install contractions

import contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 15.8 MB/s eta 0:00:00


In [29]:
import contractions
contractions.fix("you're happy now")

'you are happy now'

In [41]:
from tqdm import tqdm
from nltk.stem.wordnet import WordNetLemmatizer
wordnet = WordNetLemmatizer()

preprocessed_reviews  = []

# tqdm is for providing the status bar

for sentence in tqdm(final['Text'].values):
    sentence = re.sub(r"http\S+","",sentence)
    sentence = contractions.fix(sentence)
    sentence = BeautifulSoup(sentence,'lxml').get_text()
    sentence = re.sub("\S*\d\S*","",sentence).strip()
    senetence =  re.sub('[^A-Za-z]+',' ',sentence)
    senetence = ' '.join(e.lower() for e in sentence.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(senetence.strip())

100%|██████████| 23953/23953 [00:15<00:00, 1517.30it/s]


All cannot be controlled , but as the weird spellings anomally so it will occur less thats why we shoould not worry .

Countvectorizer will pick the top 5000 words in term of occurence , so this will likely be left out by the model.

## Feature Extraction

In [44]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(binary=True ,
                            max_features=5000 ,
                            ngram_range=(1,2))

count_vect.fit(preprocessed_reviews)

CountVectorizer(binary=True, max_features=5000, ngram_range=(1, 2))

In [45]:
print("Some Features name :",count_vect.get_feature_names_out()[:20])
print("="*50)

final_counts = count_vect.transform(preprocessed_reviews)

print("Type of Count Vectorizer : " , type(final_counts))
print("The shape of BOW : " , final_counts.get_shape())

Some Features name : ['ability' 'able' 'able buy' 'able find' 'able get' 'about' 'above'
 'absolute' 'absolute favorite' 'absolutely' 'absolutely best'
 'absolutely delicious' 'absolutely love' 'absolutely loved'
 'absolutely loves' 'absolutely no' 'acceptable' 'accident' 'according'
 'accustomed']
Type of Count Vectorizer :  <class 'scipy.sparse._csr.csr_matrix'>
The shape of BOW :  (23953, 5000)


final_counts is a sparse vector containing values if the words exist or not in the records

## TF-IDF

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf_vect = TfidfVectorizer(ngram_range=(1,2),
                              max_features=5000)
tf_idf_vect.fit(preprocessed_reviews)

print("Some feature name : ", tf_idf_vect.get_feature_names_out()[:20])
print('='*50)

final_counts = tf_idf_vect.transform(preprocessed_reviews)

print("The type of count vectorizer",type(final_counts))
print("The shape of out text BOW vectorizer",final_counts.get_shape())
print('***'*100)
print()

Some feature name :  ['ability' 'able' 'able find' 'able get' 'about' 'above' 'absolute'
 'absolute favorite' 'absolutely' 'absolutely delicious' 'absolutely love'
 'absolutely loved' 'absolutely loves' 'absolutely no' 'acceptable'
 'according' 'acerola' 'acid' 'acid coffee' 'acid reflux']
The type of count vectorizer <class 'scipy.sparse._csr.csr_matrix'>
The shape of out text BOW vectorizer (23953, 5000)
************************************************************************************************************************************************************************************************************************************************************************************************************
   0     1     2     3     4     5     6     7     8     9     ...  4990  \
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0

In [51]:
pd.DataFrame(final_counts.toarray()).head()

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
